In [1]:
import os
import pandas as pd

SCORE_DIR = "data/scores"

In [2]:
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [3]:
from bs4 import BeautifulSoup

def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [4]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [5]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    cols = [c.lower() for c in cols]
    line_score.columns = cols
    return line_score

In [6]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [32]:
games = []
base_cols = None
for box_score in box_scores:
    soup = parse_html(box_score)

    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
        
        cols = [c for c in list(summary.index) if c in base_cols]
        summary = summary[cols]
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    
    full_game["ot_game"] = ("ot" in list(summary.columns))
    
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 8886
200 / 8886


KeyboardInterrupt: 

In [50]:
games_df = pd.concat(games[:7], ignore_index=True, axis=0)

In [54]:
set(games[7].columns) - set(games[8].columns)

{'ot', 'ot_opp'}

In [46]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,team_opp,1_opp,2_opp,3_opp,4_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,DAL,23,21,28,23,95,1,2016,2015-12-09,True
1,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,...,ATL,22,27,27,22,98,0,2016,2015-12-09,False
2,240.0,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,...,SAS,32,21,34,20,107,1,2018,2017-10-18,False
3,240.0,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,...,MIN,25,24,28,22,99,0,2018,2017-10-18,True
4,240.0,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,...,MEM,19,20,36,17,92,1,2021,2021-04-30,False
5,240.0,240.0,34.0,99.0,0.343,11.0,39.0,0.282,13.0,16.0,...,ORL,20,23,15,17,75,0,2021,2021-04-30,True
6,240.0,240.0,35.0,83.0,0.422,9.0,24.0,0.375,12.0,18.0,...,MIN,31,27,24,25,107,1,2017,2017-03-08,False
7,240.0,240.0,42.0,86.0,0.488,5.0,20.0,0.250,18.0,20.0,...,LAC,19,36,13,23,91,0,2017,2017-03-08,True
8,240.0,240.0,44.0,87.0,0.506,10.0,23.0,0.435,27.0,32.0,...,DAL,23,33,33,32,121,1,2019,2019-03-26,True
9,240.0,240.0,39.0,77.0,0.506,15.0,45.0,0.333,28.0,32.0,...,SAC,25,29,33,38,125,0,2019,2019-03-26,False


In [31]:
summary.index.drop_duplicates(keep="first")

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%', 'orb',
       'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', '+/-', 'ts%',
       'efg%', '3par', 'ftr', 'orb%', 'drb%', 'trb%', 'ast%', 'stl%', 'blk%',
       'tov%', 'usg%', 'ortg', 'drtg', 'unnamed: 16', 'mp_max', 'fg_max',
       'fga_max', 'fg%_max', '3p_max', '3pa_max', '3p%_max', 'ft_max',
       'fta_max', 'ft%_max', 'orb_max', 'drb_max', 'trb_max', 'ast_max',
       'stl_max', 'blk_max', 'tov_max', 'pf_max', 'pts_max', '+/-_max',
       'ts%_max', 'efg%_max', '3par_max', 'ftr_max', 'orb%_max', 'drb%_max',
       'trb%_max', 'ast%_max', 'stl%_max', 'blk%_max', 'tov%_max', 'usg%_max',
       'ortg_max', 'drtg_max', 'unnamed: 16_max'],
      dtype='object')